In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


env_path = os.path.join(os.getcwd(), "config", ".env")

_ = load_dotenv(dotenv_path=env_path)


api_key = os.getenv("OPENAI_API_KEY")

if not api_key: 
	raise ValueError("API key missing")

Some key parameters are explained here:
- **os.path.join(os.getcwd(), "config", ".env")** constructs the full path to the `.env` file.
- **load_dotenv(dotenv_path=env_path)** loads environment variables from the `.env` file at the specified path into the script’s runtime. This allows secure access to sensitive keys.
- **os.getenv("OPENAI_API_KEY")** fetches the value of `OPENAI_API_KEY` from the environment variables loaded by load_dotenv. Returns `None` if the key is missing.
- **ChatOpenAI()** creates an instance of OpenAI’s chat model via LangChain, configured to interact with the API.
	- `model` specifies the model variant to use.
	- `temperature` controls randomness in responses, from `0` (predictable, deterministic outputs) to `1` (highly creative/random).
	- `top_p` nucleus sampling, which limits token selection to the top 70% (0.7 in this example) of probable options. Works with `temperature` to fine-tune response diversity.


In [2]:
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.5,
    top_p=0.7,
)

In [ ]:
prompt = "Explain black holes in one sentence."
response = llm.invoke(prompt)
print(f"Prompt: {prompt}\n")
print(f"Response: {response}\n")

In [ ]:
prompt = """Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:
"""
response = llm.invoke(prompt)
print(f"Prompt: {prompt}\n")
print(f"Response: {response.content}\n")

In [ ]:
template = """Answer the question as if you were {role}.
QUESTION: {usr_query}
"""
prompt = PromptTemplate.from_template(template)
print(prompt.format(usr_query="Tell me about black holes.", role="Richard Feynman"))

In [ ]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke({"role": "Richard Feynman", "usr_query": "Tell me about black holes."})
print(response)

In [ ]:
content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

template = """Summarize the {content} in one sentence.
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
summarize_chain = (
    prompt
    | llm 
    | StrOutputParser()
)

# Run the chain
summary = summarize_chain.invoke({"content": content})
print(summary)

In [3]:
role = """
    Dungeon & Dragons game master
"""

tone = "engaging and immersive"

template = """
    You are an expert {role}. I have this question {question}. I would like our conversation to be {tone}.
    
    Answer:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
roleplay_chain = (
    prompt
    | llm 
    | StrOutputParser()
)

while True:
    # Get User input
    user_message = input("\nUser: ")
    
    if user_message.lower() == "exit":
        break


    # Pass user prompt to generate Response
    response = roleplay_chain.invoke({"role": role, "question": user_message, "tone": tone})
    
    print("\nAI Message: ", response)


AI Message:  Greetings, brave adventurer! I am at your service as your Dungeon Master, ready to weave tales of heroism, mystery, and danger. What quest or conundrum shall we embark upon today? Whether you seek guidance on character creation, world-building, or a thrilling encounter, I stand ready to assist. Tell me, what stirs your imagination?

AI Message:  Ah, a fellow seeker of adventure! Pray, what question stirs your curiosity in the realms of Dungeons & Dragons? Whether it be the weaving of intricate lore, the crafting of perilous encounters, or the secrets of character mastery, I stand ready to guide you through the mists of imagination. Speak, and let our tale begin!

AI Message:  Ah, brave adventurer, the winds of fate have whispered of many paths awaiting your footsteps today. The sun rises over the ancient kingdom of Eldoria, where shadows lengthen and secrets stir beneath the cobblestones. Will you delve into the forgotten catacombs beneath the ruined keep, rumored to be h

In [9]:
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

template = """
Analyze the following product review:
"{review}"

Provide your analysis in the following format:
- Sentiment: (positive, negative, or neutral)
- Key Features Mentioned: (list the product features mentioned)
- Summary: (one-sentence summary)
"""

prompt = PromptTemplate.from_template(template)

#chain = prompt | llm | StrOutputParser()

def format_review_prompt(variables):
    # The variables parameter expects a dictionary with all template placeholders
    # In this case, just {"review": "the actual review text"}
    return prompt.format(**variables)

chain = RunnableLambda(format_review_prompt) | llm | StrOutputParser()

for i, review in enumerate(reviews):
	res = chain.invoke({"review": review})
	print(f"Review: {i + 1}")
	print(f"\n{res}\n")

Review: 1

- Sentiment: Positive  
- Key Features Mentioned: Camera quality, battery life, heating during gaming  
- Summary: The reviewer is very satisfied with the smartphone's camera and battery performance, though they note it tends to heat up slightly during gaming.

Review: 2

- Sentiment: Negative  
- Key Features Mentioned: Speed, stability (crashes), keyboard functionality, customer service  
- Summary: The laptop performs poorly with slow speed, frequent crashes, a faulty keyboard after two months, and unhelpful customer service.



#### Basic Chain Approach:

```python
chain = product_review_prompt | llm | StrOutputParser()
```

This works perfectly when you have simple variables to pass to the prompt since the prompt template automatically handles the formatting.

#### RunnableLambda Approach:

```python
def format_review_prompt(variables):
    # The variables parameter expects a dictionary with all template placeholders
    # In this case, just {"review": "the actual review text"}
    return product_review_prompt.format(**variables)

chain = RunnableLambda(format_review_prompt) | llm | StrOutputParser()
```

Use when you need to:
  - Preprocess input variables before formatting
  - Handle complex input structures
  - Add conditional logic to prompt generation

If you need to preprocess:

```python
# If you get input like this:
raw_input = "This product is amazing! The battery lasts forever."

def format_review_prompt(raw_text):
    # Clean the input first
    cleaned = raw_text.strip().replace("!", "")
    return product_review_prompt.format(review=cleaned)
```

Use your normal `prompt | llm | output_parser` chain for simple cases and only use `RunnableLambda` when you need to:
- Transform the input structure
- Add preprocessing logic
- Handle complex variable mapping

In thjs specific example, the `format_review_prompt` function is redundant unless you need to modify the input.